# SET CODING ENVIRONMENT

## Module Imports

In [1]:
from pprint import pprint
import ee
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta
import folium
from folium import plugins
import matplotlib.pyplot as plt
from pathlib import Path
import ast


## Authenticate to Google Earth Engine

In [2]:
ee.Authenticate() #Uncomment this whenever needed, once done usually not needed for 1-2 days
ee.Initialize(project='ee-raman')

# FUNCTION DEFINITIONS

### My Functions

In [3]:
def get_feature_from_collection(collec, ind):
    """
    Sometimes directly getting ith feature form collection doesn't work
    This hack works all the time. Converting collection to list and getting
    ith element and re-casting it as feature.
    """
    return ee.Feature(collec.toList(collec.size().getInfo()).get(ind))

## Common Function Definitions

In [4]:
def displayMap(roi_boundary, image):
  centroid = roi_boundary.geometry().centroid()
  coordinates = centroid.coordinates()
  centerLat = coordinates.get(1).getInfo()
  centerLon = coordinates.get(0).getInfo()
  mapObj = folium.Map(width='100%', height='80%', location=[centerLat, centerLon], zoom_start=50)

  # Sattelite image visual parameters
  vis_params = {
    'min': 0,
    'max': 12,
    'palette': ['#000000',  # 0 Black- background
              '#ff0000',   # 1 Red- builtup
              '#74ccf4', # 2 Light Blue- kharif water
              '#1ca3ec', # 3 Blue- kharif and rabi water
              '#0f5e9c', # 4 Dark Blue- kharif and rabi and zaid water
              '#f1c232', # 5 Yellow- croplands
              '#38761d', # 6 Dark Green- Tree/Forests
              '#A9A9A9', # 7 Gray- barren lands
              '#f1c232', # 8 Yellow- Single Kharif Cropping
              '#f59d22', # 9 Mustard- Single Non-Kharif Cropping
              '#e68600', # 10 Orange- Double Cropping
              '#b3561d', # 11 Brown- Triple Cropping
              '#c39797' # 12 Mauve- Shrubs_Scrubs
            ]
    }

  map_id_dict = ee.Image(image).getMapId(vis_params)

  folium.TileLayer(
          tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
          attr='Esri',
          name='Esri Satellite',
          overlay=True,
          control=True
      ).add_to(mapObj)

  folium.raster_layers.TileLayer(
                  tiles = map_id_dict['tile_fetcher'].url_format,
                  attr = 'Google Earth Engine',
                  name = 'Sentinel 2 image',
                  overlay = True,
                  control = True
                  ).add_to(mapObj)

  mapObj.add_child(folium.LayerControl())
  display(mapObj)


'''
Function to mask clouds based on the QA60 band of Sentinel SR data.
param {ee.Image} image Input Sentinel SR image
return {ee.Image} Cloudmasked Sentinel-2 image
'''
def maskS2cloud(image):
  qa = image.select('QA60')
  #Bits 10 and 11 are clouds and cirrus, respectively.
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11
  #Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
  return image.updateMask(mask).divide(10000)

## Cropping Frequency Detection

In [5]:
chastainBandNames = ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2']

# Regression model parameters from Table-4. MSI TOA reflectance as a function of OLI TOA reflectance.
msiOLISlopes = [1.0946,1.0043,1.0524,0.8954,1.0049,1.0002]
msiOLIIntercepts = [-0.0107,0.0026,-0.0015,0.0033,0.0065,0.0046]

# Regression model parameters from Table-5. MSI TOA reflectance as a function of ETM+ TOA reflectance.
msiETMSlopes = [1.10601,0.99091,1.05681,1.0045,1.03611,1.04011]
msiETMIntercepts = [-0.0139,0.00411,-0.0024,-0.0076,0.00411,0.00861]

# Regression model parameters from Table-6. OLI TOA reflectance as a function of ETM+ TOA reflectance.
oliETMSlopes =[1.03501,1.00921,1.01991,1.14061,1.04351,1.05271];
oliETMIntercepts = [-0.0055,-0.0008,-0.0021,-0.0163,-0.0045,0.00261]

# Construct dictionary to handle all pairwise combos
chastainCoeffDict = { 'MSI_OLI':[msiOLISlopes,msiOLIIntercepts,1], # check what the third item corresponds to
                      'MSI_ETM':[msiETMSlopes,msiETMIntercepts,1],
                      'OLI_ETM':[oliETMSlopes,oliETMIntercepts,1],
                      'OLI_MSI':[msiOLISlopes,msiOLIIntercepts,0],
                      'ETM_MSI':[msiETMSlopes,msiETMIntercepts,0],
                      'ETM_OLI':[oliETMSlopes,oliETMIntercepts,0]
                    }


'''
Function to mask cloudy pixels in Landsat-7
'''
def maskL7cloud(image):
  qa = image.select('QA_PIXEL')
  mask = qa.bitwiseAnd(1 << 4).eq(0)
  return image.updateMask(mask).select(['B1', 'B2', 'B3' , 'B4' , 'B5' , 'B7']).rename('BLUE', 'GREEN', 'RED' , 'NIR' , 'SWIR1' , 'SWIR2')


'''
Function to mask cloudy pixels in Landsat-8
'''
def maskL8cloud(image):
  qa = image.select('QA_PIXEL')
  mask = qa.bitwiseAnd(1 << 4).eq(0)
  return image.updateMask(mask).select(['B2', 'B3', 'B4' , 'B5' , 'B6' , 'B7']).rename('BLUE', 'GREEN', 'RED' , 'NIR' , 'SWIR1' , 'SWIR2')


'''
Function to mask clouds using the quality band of Sentinel-2 TOA
'''
def maskS2cloudTOA(image):
  qa = image.select('QA60')
  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11
  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0));
  return image.updateMask(mask).select(['B2', 'B3', 'B4', 'B8',  'B11', 'B12']).rename(['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2'])


'''
Get Landsat and Sentinel image collections
'''
def Get_L7_L8_S2_ImageCollections(inputStartDate, inputEndDate, roi_boundary):
  # ------ Landsat 7 TOA
  L7 = ee.ImageCollection('LANDSAT/LE07/C02/T1_TOA') \
          .filterDate(inputStartDate, inputEndDate) \
          .filterBounds(roi_boundary) \
          .map(maskL7cloud)
  # print('\n Original Landsat 7 TOA dataset: \n',L7.limit(1).getInfo())
  # print('Number of images in Landsat 7 TOA dataset: \t',L7.size().getInfo())

  # ------ Landsat 8 TOA
  L8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA') \
          .filterDate(inputStartDate, inputEndDate) \
          .filterBounds(roi_boundary) \
          .map(maskL8cloud)
  # print('\n Original Landsat 8 TOA dataset: \n', L8.limit(1).getInfo())
  # print('Number of images in Landsat 8 TOA dataset: \t',L8.size().getInfo())

  # ------ Sentinel-2 TOA
  S2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
          .filterDate(inputStartDate, inputEndDate) \
          .filterBounds(roi_boundary)  \
          .map(maskS2cloudTOA)
  # print('\n Original Sentinel-2 TOA dataset: \n',S2.limit(1).getInfo())
  # print('Number of images in Sentinel 2 TOA dataset: \t',S2.size().getInfo())

  return L7, L8, S2


'''
Function to apply model in one direction
'''
def dir0Regression(img,slopes,intercepts):
  return img.select(chastainBandNames).multiply(slopes).add(intercepts)


'''
Applying the model in the opposite direction
'''
def dir1Regression(img,slopes,intercepts):
  return img.select(chastainBandNames).subtract(intercepts).divide(slopes)


'''
Function to correct one sensor to another
'''
def harmonizationChastain(img, fromSensor,toSensor):
  # Get the model for the given from and to sensor
  comboKey = fromSensor.upper() + '_' + toSensor.upper()
  coeffList = chastainCoeffDict[comboKey]
  slopes = coeffList[0]
  intercepts = coeffList[1]
  direction = ee.Number(coeffList[2])

  # Apply the model in the respective direction
  out = ee.Algorithms.If(direction.eq(0),dir0Regression(img,slopes,intercepts),dir1Regression(img,slopes,intercepts))
  return ee.Image(out).copyProperties(img).copyProperties(img,['system:time_start'])


'''
Calibrate Landsat-8 (OLI) and Sentinel-2 (MSI) to Landsat-7 (ETM+)
'''
def Harmonize_L7_L8_S2(L7, L8, S2):
  # harmonization
  harmonized_L8 = L8.map( lambda img: harmonizationChastain(img, 'OLI','ETM') )
  harmonized_S2 = S2.map( lambda img: harmonizationChastain(img, 'MSI','ETM') )

  # Merge harmonized landsat-8 and sentinel-2 to landsat-7 image collection
  harmonized_LandsatSentinel_ic = ee.ImageCollection(L7.merge(harmonized_L8).merge(harmonized_S2))
  # print(harmonized_LandsatSentinel_ic.size().getInfo())
  return harmonized_LandsatSentinel_ic


'''
Add NDVI band to harmonized image collection
'''
def addNDVI(image):
  return image.addBands(image.normalizedDifference(['NIR', 'RED']).rename('NDVI')).float()


'''
Function definitions to get NDVI values at each 16-day composites
'''
def Get_NDVI_image_datewise(harmonized_LS_ic):
  def get_NDVI_datewise(date):
    return harmonized_LS_ic.select(['NDVI']) \
                            .filterDate(ee.Date(date), ee.Date(date).advance(16, 'day')) \
                            .median() \
                            .set('system:time_start',ee.Date(date).millis())
  return get_NDVI_datewise

def Get_LS_16Day_NDVI_TimeSeries(inputStartDate, inputEndDate, harmonized_LS_ic):
  startDate = datetime.strptime(inputStartDate,"%Y-%m-%d")
  endDate = datetime.strptime(inputEndDate,"%Y-%m-%d")

  date_list = pd.date_range(start=startDate, end=endDate, freq='16D').tolist()
  date_list = ee.List( [datetime.strftime(curr_date,"%Y-%m-%d") for curr_date in date_list] )

  LSC =  ee.ImageCollection.fromImages(date_list.map(Get_NDVI_image_datewise(harmonized_LS_ic)))

  return LSC


'''
Pair available LSC and modis values for each time stamp.
'''
def pairLSModis(lsRenameBands):
  def pair(feature):
    date = ee.Date( feature.get('system:time_start') )
    startDateT = date.advance(-8,'day')
    endDateT = date.advance(8,'day')

    # ------ MODIS VI ( We can add EVI to the band list later )
    modis = ee.ImageCollection('MODIS/061/MOD13Q1') \
              .filterDate(startDateT, endDateT) \
              .select(['NDVI','SummaryQA']) \
              .filterBounds(roi_boundary) \
              .median() \
              .rename(['NDVI_modis', 'SummaryQA_modis'])

    return feature.rename(lsRenameBands).addBands(modis)
  return pair


'''
Function to get Pearson Correlation Coffecient to perform GapFilling
'''
def get_Pearson_Correlation_Coefficients(LSC_modis_paired_ic, roi_boundary, bandList):
  corr = LSC_modis_paired_ic.filterBounds(roi_boundary) \
                            .select(bandList).toArray() \
                            .arrayReduce( reducer = ee.Reducer.pearsonsCorrelation(), axes=[0], fieldAxis=1 ) \
                            .arrayProject([1]).arrayFlatten([['c', 'p']])
  return corr


'''Use print(...) to write to this console.
Fill gaps in LSC timeseries using modis data
'''
def gapfillLSM(LSC_modis_regression_model, LSC_bandName, modis_bandName):
  def peformGapfilling(image):
    offset = LSC_modis_regression_model.select('offset')
    scale = LSC_modis_regression_model.select('scale')
    nodata = -1

    lsc_image = image.select(LSC_bandName)
    modisfit = image.select(modis_bandName).multiply(scale).add(offset)

    mask = lsc_image.mask()#update mask needs an input (no default input from the API document)
    gapfill = lsc_image.unmask(nodata)
    gapfill = gapfill.where(mask.Not(), modisfit)

    '''
    in SummaryQA,
    0: Good data, use with confidence
    1: Marginal data, useful but look at detailed QA for more information
    2: Pixel covered with snow/ice
    3: Pixel is cloudy
    '''
    qc_m = image.select('SummaryQA_modis').unmask(3)  # missing value is grouped as cloud
    w_m  = modisfit.mask().rename('w_m').where(qc_m.eq(0), 0.8)  # default is 0.8
    w_m = w_m.where(qc_m.eq(1), 0.5)   # Marginal
    w_m = w_m.where(qc_m.gte(2), 0.2) # snow/ice or cloudy

    # make sure these modis values are read where there is missing data from LandSat, Sentinel
    w_l = gapfill.mask() # default is 1
    w_l = w_l.where(mask.Not(), w_m)

    return gapfill.addBands(w_l).rename(['gapfilled_'+LSC_bandName,'SummaryQA']) #have NDVI from modis and a summary of clarity for each

  return peformGapfilling


'''
Function to combine LSC with Modis data
'''
def Combine_LS_Modis(LSC):
  lsRenameBands = ee.Image(LSC.first()).bandNames().map( lambda band: ee.String(band).cat('_lsc') )
  LSC_modis_paired_ic = LSC.map( pairLSModis(lsRenameBands) )

  # Output contains scale, offset i.e. two bands
  LSC_modis_regression_model_NDVI = LSC_modis_paired_ic.select(['NDVI_modis', 'NDVI_lsc']) \
                                                        .reduce(ee.Reducer.linearFit())

  corr_NDVI = get_Pearson_Correlation_Coefficients(LSC_modis_paired_ic, roi_boundary, ['NDVI_modis', 'NDVI_lsc'])
  LSMC_NDVI = LSC_modis_paired_ic.map(gapfillLSM(LSC_modis_regression_model_NDVI, 'NDVI_lsc', 'NDVI_modis'))

  return LSMC_NDVI


'''
Mask out low quality pixels
'''
def mask_low_QA(lsmc_image):
  low_qa = lsmc_image.select('SummaryQA').neq(0.2)
  return lsmc_image.updateMask(low_qa).copyProperties(lsmc_image, ['system:time_start'])


'''
Add image timestamp to each image in time series
'''
def add_timestamp(image):
  timeImage = image.metadata('system:time_start').rename('timestamp')
  timeImageMasked = timeImage.updateMask(image.mask().select(0))
  return image.addBands(timeImageMasked)


'''
Perform linear interpolation on missing values
'''
def performInterpolation(image):
  image = ee.Image(image)
  beforeImages = ee.List(image.get('before'))
  beforeMosaic = ee.ImageCollection.fromImages(beforeImages).mosaic()
  afterImages = ee.List(image.get('after'))
  afterMosaic = ee.ImageCollection.fromImages(afterImages).mosaic()

  # Interpolation formula
  # y = y1 + (y2-y1)*((t – t1) / (t2 – t1))
  # y = interpolated image
  # y1 = before image
  # y2 = after image
  # t = interpolation timestamp
  # t1 = before image timestamp
  # t2 = after image timestamp

  t1 = beforeMosaic.select('timestamp').rename('t1')
  t2 = afterMosaic.select('timestamp').rename('t2')
  t = image.metadata('system:time_start').rename('t')
  timeImage = ee.Image.cat([t1, t2, t])
  timeRatio = timeImage.expression('(t - t1) / (t2 - t1)', {
                  't': timeImage.select('t'),
                  't1': timeImage.select('t1'),
                  't2': timeImage.select('t2'),
              })

  interpolated = beforeMosaic.add((afterMosaic.subtract(beforeMosaic).multiply(timeRatio)))
  result = image.unmask(interpolated)
  fill_value = ee.ImageCollection([beforeMosaic, afterMosaic]).mosaic()
  result = result.unmask(fill_value)

  return result.copyProperties(image, ['system:time_start'])


def interpolate_timeseries(S1_TS):
  lsmc_masked = S1_TS.map(mask_low_QA)
  filtered = lsmc_masked.map(add_timestamp)

  # Time window in which we are willing to look forward and backward for unmasked pixel in time series
  timeWindow = 120

  # Define a maxDifference filter to find all images within the specified days. Convert days to milliseconds.
  millis = ee.Number(timeWindow).multiply(1000*60*60*24)
  # Filter says that pick only those timestamps which lie between the 2 timestamps not more than millis difference apart
  maxDiffFilter = ee.Filter.maxDifference(
                              difference = millis,
                              leftField = 'system:time_start',
                              rightField = 'system:time_start',
                            )

  # Filter to find all images after a given image. Compare the image's timstamp against other images.
  # Images ahead of target image should have higher timestamp.
  lessEqFilter = ee.Filter.lessThanOrEquals(
                            leftField = 'system:time_start',
                            rightField = 'system:time_start'
                          )

  # Similarly define this filter to find all images before a given image
  greaterEqFilter = ee.Filter.greaterThanOrEquals(
                            leftField = 'system:time_start',
                            rightField = 'system:time_start'
                          )

  # Apply first join to find all images that are after the target image but within the timeWindow
  filter1 = ee.Filter.And( maxDiffFilter, lessEqFilter )
  join1 = ee.Join.saveAll(
                  matchesKey = 'after',
                  ordering = 'system:time_start',
                  ascending = False
          )
  join1Result = join1.apply(
                  primary = filtered,
                  secondary = filtered,
                  condition = filter1
                )

  # Apply first join to find all images that are after the target image but within the timeWindow
  filter2 = ee.Filter.And( maxDiffFilter, greaterEqFilter )
  join2 = ee.Join.saveAll(
                  matchesKey = 'before',
                  ordering = 'system:time_start',
                  ascending = True
          )
  join2Result = join2.apply(
                  primary = join1Result,
                  secondary = join1Result,
                  condition = filter2
                )

  interpolated_S1_TS = ee.ImageCollection(join2Result.map(performInterpolation))

  return interpolated_S1_TS


'''
Function Definition to get Padded NDVI LSMC timeseries image for a given ROI
'''
def Get_Padded_NDVI_TS_Image(startDate, endDate, roi_boundary):
  L7, L8, S2 = Get_L7_L8_S2_ImageCollections(startDate, endDate, roi_boundary)

  harmonized_LS_ic = Harmonize_L7_L8_S2(L7, L8, S2)
  harmonized_LS_ic = harmonized_LS_ic.map(addNDVI)
  LSC = Get_LS_16Day_NDVI_TimeSeries(startDate, endDate, harmonized_LS_ic)
  LSMC_NDVI = Combine_LS_Modis(LSC)
  Interpolated_LSMC_NDVI = interpolate_timeseries(LSMC_NDVI)
  final_LSMC_NDVI_TS = Interpolated_LSMC_NDVI.select(['gapfilled_NDVI_lsc']).toBands()
  final_LSMC_NDVI_TS = final_LSMC_NDVI_TS.clip(roi_boundary)
  return final_LSMC_NDVI_TS

def get_cropping_frequency(roi_boundary, startDate, endDate):
  final_LSMC_NDVI_TS =  Get_Padded_NDVI_TS_Image(startDate, endDate, roi_boundary)
  return final_LSMC_NDVI_TS

# MAIN FUNCTION

## Take user input on ROI

In [6]:
top_left = [10.62060477, 77.87217496]  # Replace lon1 and lat1 with actual values
bottom_right = [10.57740888, 77.91612074]  # Replace lon2 and lat2 with actual values

# Create a rectangle geometry using the defined corners
rectangle = ee.Geometry.Rectangle([top_left[1], bottom_right[0], bottom_right[1], top_left[0]])
print("Area of the Rectangle is ", rectangle.area().getInfo()/1e6)
# Create a feature collection with the rectangle as a boundary
roi_boundary = ee.FeatureCollection([ee.Feature(rectangle)])
directory = 'Area_testing_1'


Area of the Rectangle is  23.070494296156163


In [7]:
top_left = [20.56149133, 73.56330458]  # Replace lon1 and lat1 with actual values
bottom_right = [20.47917109, 73.60725362]  # Replace lon2 and lat2 with actual values

# Create a rectangle geometry using the defined corners
rectangle = ee.Geometry.Rectangle([top_left[1], bottom_right[0], bottom_right[1], top_left[0]])
print("Area of the Rectangle is ", rectangle.area().getInfo()/1e6)
# Create a feature collection with the rectangle as a boundary
roi_boundary = ee.FeatureCollection([ee.Feature(rectangle)])
directory = 'Area_surgana'


Area of the Rectangle is  41.89448898377194


In [8]:
top_left = [19.26903317, 80.86453702]  # Replace lon1 and lat1 with actual values
bottom_right = [19.24167092, 80.89408520]  # Replace lon2 and lat2 with actual values 

# Create a rectangle geometry using the defined corners
rectangle = ee.Geometry.Rectangle([top_left[1], bottom_right[0], bottom_right[1], top_left[0]])
print("Area of the Rectangle is ", rectangle.area().getInfo()/1e6)
# Create a feature collection with the rectangle as a boundary
roi_boundary = ee.FeatureCollection([ee.Feature(rectangle)])
directory = 'Area_forested'


Area of the Rectangle is  9.437397485806445


In [9]:
top_left = [19.26903317, 80.86453702]  # Replace lon1 and lat1 with actual values
bottom_right = [19.24167092, 80.89408520]  # Replace lon2 and lat2 with actual values  
# Create a rectangle geometry using the defined corners
rectangle = ee.Geometry.Rectangle([top_left[1], bottom_right[0], bottom_right[1], top_left[0]])
print("Area of the Rectangle is ", rectangle.area().getInfo()/1e6)
# Create a feature collection with the rectangle as a boundary
roi_boundary = ee.FeatureCollection([ee.Feature(rectangle)])

directory = 'Area_plantation'


Area of the Rectangle is  9.437397485806445


In [10]:
roi_boundary = ee.FeatureCollection("users/mtpictd/india_block_boundaries").filter(ee.Filter.eq("block", "Bheramgarh"))
directory = "Area_bheramgarh"
roi_boundary = ee.FeatureCollection("users/mtpictd/india_block_boundaries").filter(ee.Filter.eq("block", "Jamkhed"))
directory = "Area_jamkhed"
roi = ee.FeatureCollection("users/mtpictd/india_block_boundaries").filter(ee.Filter.eq("block", "Peddapally"))
directory = "Area_Peddapally"

In [11]:
import math
from itertools import product


# Function to convert latitude to pixel Y at a given zoom level
def lat_to_pixel_y(lat, zoom):
    sin_lat = math.sin(math.radians(lat))
    pixel_y = ((0.5 - math.log((1 + sin_lat) / (1 - sin_lat)) / (4 * math.pi)) * (2 ** (zoom + 8)))
    return pixel_y

# Function to convert longitude to pixel X at a given zoom level
def lon_to_pixel_x(lon, zoom):
    pixel_x = ((lon + 180) / 360) * (2 ** (zoom + 8))
    return pixel_x

# Function to convert pixel X to longitude
def pixel_x_to_lon(pixel_x, zoom):
    lon = (pixel_x / (2 ** (zoom + 8))) * 360 - 180
    return lon

# Function to convert pixel Y to latitude
def pixel_y_to_lat(pixel_y, zoom):
    n = math.pi - 2 * math.pi * pixel_y / (2 ** (zoom + 8))
    lat = math.degrees(math.atan(math.sinh(n)))
    return lat

def lat_lon_from_pixel(lat, lon, zoom, scale):
    """
    Given a starting latitude and longitude, calculate the latitude and longitude
    of the opposite corner of a 256x256 image at a given zoom level.
    """
    pixel_x = lon_to_pixel_x(lon, zoom)
    pixel_y = lat_to_pixel_y(lat, zoom)
    
    new_lon = pixel_x_to_lon(pixel_x + 256*scale, zoom)
    new_lat = pixel_y_to_lat(pixel_y + 256*scale, zoom)

    return new_lat, new_lon

"""

Helper function for dividing an roi into blocks

"""
def get_n_boxes(lat, lon, n, zoom, scale):
    diagonal_lat_lon = [(lat, lon),]
    for i in range(n):
        new_lat_lon = lat_lon_from_pixel(lat, lon, zoom, scale)
        diagonal_lat_lon.append(new_lat_lon)
        lat, lon = new_lat_lon
    lats = [i[0] for i in diagonal_lat_lon]
    longs = [i[1] for i in diagonal_lat_lon]
    return list(product(lats, longs))

def get_points(roi, directory):
    points_file = Path(directory + "/status.csv")
    if points_file.is_file():
        df = pd.read_csv(directory + "/status.csv", index_col=False)
        df["points"] = df['points'].apply(ast.literal_eval)
        return df
    zoom = 17
    scale = 16
    bounds = roi.bounds().coordinates().get(0).getInfo()
    lons = sorted([i[0] for i in bounds])
    lats = sorted([i[1] for i in bounds])
    starting_point = lats[-1], lons[0]
    min_, max_ = (
        [lon_to_pixel_x(lons[0], zoom), lat_to_pixel_y(lats[0], zoom) ],
        [lon_to_pixel_x(lons[-1], zoom), lat_to_pixel_y(lats[-1], zoom)]
        )
    iterations = math.ceil(max(abs(min_[0] -  max_[0]), abs(min_[1] - max_[1]))/256/16)
    points = get_n_boxes(starting_point[0], starting_point[1], iterations, zoom, scale)
    intersect_list = []
    print(len(points))
    index = 0
    for point in points:
        top_left = point
        bottom_right = lat_lon_from_pixel(top_left[0], top_left[1], zoom, scale)
        rectangle = ee.Geometry.Rectangle([(top_left[1], top_left[0]), (bottom_right[1], bottom_right[0])])
        print(top_left, bottom_right)
        intersects = roi.geometry().intersects(rectangle, ee.ErrorMargin(1)).getInfo()
        if intersects:
            intersect_list.append((index, (top_left,bottom_right)))
            index+=1
        print(intersects)
    df = pd.DataFrame(intersect_list, columns=["index", "points"])
    df["overall_status"] = False
    df["download_status"] = False
    df["model_status"] = False
    df["segmentation_status"] = False
    df["postprocessing_status"] = False
    df["plantation_status"] = False
    df.to_csv(directory + "/status.csv", index=False)
    return df


blocks_df = get_points(roi_boundary, directory)
points = list(blocks_df["points"])

roi_boundary = ee.FeatureCollection([ee.Feature(ee.Geometry.Rectangle([top_left[1], bottom_right[0], bottom_right[1], top_left[0]])) for top_left, bottom_right in points])


#### Visualize roi on maps

## LULC execution for years 2017 onwards with temporal correction

In [12]:
startDate = '2023-07-01'
endDate = '2024-07-01'

scale = 10

loopStart = startDate
loopEnd = (datetime.strptime(endDate,"%Y-%m-%d")).strftime("%Y-%m-%d")

while loopStart != loopEnd:
    currStartDate = datetime.strptime(loopStart,"%Y-%m-%d")
    currEndDate = (currStartDate+relativedelta(years=1)-timedelta(days=1))

    loopStart = (currStartDate+relativedelta(years=1)).strftime("%Y-%m-%d")

    currStartDate = currStartDate.strftime("%Y-%m-%d")
    currEndDate = currEndDate.strftime("%Y-%m-%d")

    print("\n EXECUTING LULC PREDICTION FOR ",currStartDate," TO ",currEndDate,"\n")

    curr_filename = directory + '_' + currStartDate + "_" + currEndDate

    if datetime.strptime(currStartDate,"%Y-%m-%d").year < 2017:
        print("To generate LULC output of year ",datetime.strptime(currStartDate,"%Y-%m-%d").year," , go to cell-LULC execution for years before 2017")
        continue
    ts_data = Get_Padded_NDVI_TS_Image(startDate, endDate, roi_boundary)


 EXECUTING LULC PREDICTION FOR  2023-07-01  TO  2024-06-30 



In [14]:
ts_data = ts_data.select(ts_data.bandNames().getInfo()).rename(["_".join(i.split("_")[1:]) + "_" + i.split("_")[0] for i in ts_data.bandNames().getInfo()])
task = ee.batch.Export.image.toAsset(
    image=ts_data.clip(roi_boundary.geometry()),
    description='Time Series' + directory,
    assetId='projects/ee-raman/assets/temp12_' + directory.split("_")[-1],
    #pyramidingPolicy = {'predicted_label': 'mode'},
    scale = scale,
    maxPixels = 1e13,
    crs = 'EPSG:4326'
)
task.start()
